# The Black Scholes Model


In the Black Scholes model, the stock follows the following process.
$$
dx_t = (\mu_t - \frac{\sigma^2}{2})dt + \sigma dW_t
$$
where,
$$
x_t = \ln S_t
$$

## PDE for the contract price

Let $V$ be the price of the contract. $V_x$ and $V_{xx}$ are the derivatives w.r.t. $x$.
$$
V_x (\mu_t - \frac{\sigma^2}{2})dt + V_{xx}\frac{\sigma^2}{2}dt + dV - r V dt = 0
$$

In [ ]:
from sympy import *
from sympy import init_printing
from utils.symfns import discretize_crank_nicolson, find_coefficients

init_printing()

dV, V, Vx, Vxx, sigma, mu, r, dx, dt, x = symbols(
    "dV V V_x V_{xx} sigma mu r dx, dt, x"
)

We will define the LHS of the equation using sympy as follows:

In [ ]:
model_expr = (
    dV + sigma**2 / 2 * Vxx * dt + (mu - sigma**2 / 2) * Vx * dt - r * V * dt
)
model_expr

                ⎛     2⎞              2     
                ⎜    σ ⎟   V_{xx}⋅dt⋅σ      
-V⋅dt⋅r + Vₓ⋅dt⋅⎜μ - ──⎟ + ──────────── + dV
                ⎝    2 ⎠        2           

## Discretize

We can now substitute $V$, $dV$, $V_x$, and $V_{xx}$ in the above PDE, by the discrete estimates based on the Crank-Nicolson method, as described in the [README](../README.md).

In [ ]:
expr, left_gridpoints, right_gridpoints = discretize_crank_nicolson(
    model_expr, V, dV, Vx, Vxx, dx
)
expr

                                               ⎛     2⎞                       
                                               ⎜    σ ⎟ ⎛  V_l__d   V_r__d   V
                                            dt⋅⎜μ - ──⎟⋅⎜- ────── - ────── + ─
                        ⎛V_l__m   V_r__m⎞      ⎝    2 ⎠ ⎝    2        2       
-V_l__m + V_r__m - dt⋅r⋅⎜────── + ──────⎟ + ──────────────────────────────────
                        ⎝  2        2   ⎠                          2⋅dx       
                                                                              

                                                                              
_l__u   V_r__u⎞                                                               
───── + ──────⎟       2                                                       
 2        2   ⎠   dt⋅σ ⋅(V_l__d + V_r__d - 2⋅V_l__m - 2⋅V_r__m + V_l__u + V_r_
─────────────── + ────────────────────────────────────────────────────────────
                                                   

We now need to find the coefficients of $V^u_p$, $V^m_p$, $V^d_p$, $V^u$, $V^m$, $V^d$

The PDE is in the form
$$
Coefs \cdot
\begin{bmatrix}
V^u_l & V^u_r \\
V^m_l & V^m_r \\
V^d_l & V^d_r
\end{bmatrix} = 0
$$

Therefore, if we re-arrange into the form:

$$
A_l \cdot \begin{bmatrix} V^u_l \\ V^m_l \\ V^d_l \end{bmatrix} =
A_r \cdot \begin{bmatrix} V^u_r \\ V^m_r \\ V^d_r \end{bmatrix}
$$

We can run backward induction, by solving for $A_l$ when $A_r$ is known.

$A_l$ is given by the left coefficients, and $A_r$ is given by the **negative** of the right coefficients. Here they are, including the special cases at the grid boundaries, with the negatives already applied.

In [ ]:
for row_type in ["normal", "up", "dn"]:
    print(f"\nRow type: {row_type}")
    expr_up, left_gridpoints, right_gridpoints = discretize_crank_nicolson(
        model_expr, V, dV, Vx, Vxx, dx, rowtype=row_type
    )
    coefs_up = find_coefficients(expr_up, left_gridpoints + right_gridpoints)

    for k, v in coefs_up.items():
        flip = -1 if k in right_gridpoints else 1
        print(f"{k}: {v * flip}")


Row type: normal
V^u_l: dt*(mu - sigma**2/2)/(4*dx) + dt*sigma**2/(4*dx**2)
V^m_l: -dt*r/2 - dt*sigma**2/(2*dx**2) - 1
V^d_l: -dt*(mu - sigma**2/2)/(4*dx) + dt*sigma**2/(4*dx**2)
V^u_r: -dt*(mu - sigma**2/2)/(4*dx) - dt*sigma**2/(4*dx**2)
V^m_r: dt*r/2 + dt*sigma**2/(2*dx**2) - 1
V^d_r: dt*(mu - sigma**2/2)/(4*dx) - dt*sigma**2/(4*dx**2)

Row type: up
V^u_l: 0
V^m_l: -dt*r/2 + dt*(mu - sigma**2/2)/(2*dx) - 1
V^d_l: -dt*(mu - sigma**2/2)/(2*dx)
V^u_r: 0
V^m_r: dt*r/2 - dt*(mu - sigma**2/2)/(2*dx) - 1
V^d_r: dt*(mu - sigma**2/2)/(2*dx)

Row type: dn
V^u_l: dt*(mu - sigma**2/2)/(2*dx)
V^m_l: -dt*r/2 - dt*(mu - sigma**2/2)/(2*dx) - 1
V^d_l: 0
V^u_r: -dt*(mu - sigma**2/2)/(2*dx)
V^m_r: dt*r/2 + dt*(mu - sigma**2/2)/(2*dx) - 1
V^d_r: 0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7b2bac78-17c7-471e-96cd-db7e0e632f5e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>